# Simulation 03 -- Asymmetric Bias Stress Test

## THE MOST DANGEROUS SCENARIO

**Question:** If Verizon systematically overpays (the biggest real-world risk), does our pipeline detect it — or silently produce biased FMV predictions?

### Why This Matters

In practice, carrier lease rents are **not** randomly noisy around true FMV. The noise is **asymmetric**:
- ~70% of sites are **overpaying** (contractual escalation compounds over 15-25 years)
- ~15% are underpaying (negotiated down at renewal)
- ~15% are roughly at market (recently renegotiated)

Our pipeline uses current rents as a starting signal (via MAD pseudo-labeling). If most rents are inflated, the pseudo-labels are biased upward, the model trains on biased targets, and the predicted "FMV" silently inherits the overpayment — **exactly what we're trying to detect**.

### What We Test

| Bias Level | Avg Overpayment | Scenario |
|:---:|:---:|:---:|
| Level 1 | +10% | Mild — early-stage leases |
| Level 2 | +20% | Moderate — typical portfolio |
| Level 3 | +30% | Severe — aged portfolio |
| Level 4 | +40% | Extreme — worst-case stress |

For each level we:
1. Inject asymmetric distortion into rents
2. Run the standard pipeline (HDBSCAN → MAD → XGBoost+QRF)
3. Measure how much bias leaks into predicted FMV
4. Test whether defense layers catch the bias
5. Test fallback corrections that reduce residual bias

In [ ]:
# ============================================================
# Imports & Setup
# ============================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, time, warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import Ridge
from sklearn.metrics import (
    silhouette_score, mean_absolute_error, r2_score,
    confusion_matrix, classification_report, cohen_kappa_score
)
from sklearn.ensemble import IsolationForest
from scipy.stats import median_abs_deviation, pearsonr
from xgboost import XGBRegressor

# HDBSCAN: try standalone package first, fall back to sklearn
try:
    from hdbscan import HDBSCAN
    print('Using standalone hdbscan package')
except ImportError:
    from sklearn.cluster import HDBSCAN
    print('Using sklearn HDBSCAN')

# Quantile forest: try quantile-forest package, fall back to sklearn RF
try:
    from quantile_forest import RandomForestQuantileRegressor
    HAS_QRF = True
    print('Using quantile-forest package')
except ImportError:
    from sklearn.ensemble import RandomForestRegressor
    HAS_QRF = False
    print('Falling back to sklearn RandomForestRegressor (no native quantiles)')

np.random.seed(42)

# Plot style
sns.set_style('whitegrid')
plt.rcParams.update({
    'figure.dpi': 120,
    'font.size': 10,
    'axes.titlesize': 12,
    'axes.labelsize': 10,
})

print('All imports successful.')

## 1. Load Synthetic Data & Define Feature Sets

In [ ]:
# ============================================================
# Load the synthetic dataset generated in sim_00
# ============================================================
DATA_PATH = os.path.join('..', 'data', 'synthetic', 'synthetic_5k_sites.parquet')
df_raw = pd.read_parquet(DATA_PATH)
print(f'Loaded {len(df_raw):,} sites, {df_raw.shape[1]} columns')
print(f'True FMV range: ${df_raw["true_fmv"].min():,.0f} – ${df_raw["true_fmv"].max():,.0f}')
print(f'True FMV median: ${df_raw["true_fmv"].median():,.0f}')

# ============================================================
# Feature lists (identical to sim_01 standard definitions)
# ============================================================
X_MARKET = [
    'census_population_density', 'census_population_3km',
    'census_median_income', 'census_median_home_value',
    'census_median_rent', 'census_vacancy_rate',
    'urban_rural_class', 'underlying_land_value_psf',
    'hud_safmr', 'property_tax_rate',
    'topography_class', 'terrain_elevation_variance_m',
    'poi_density_1km', 'building_density_500m',
    'distance_to_highway_km', 'ground_elevation_ft',
    'unemployment_rate_local',
    'fcc_tower_count_5km', 'fcc_tower_density_sqkm',
    'competitor_tower_distance_km', 'tower_per_capita',
    'scarcity_index', 'landlord_concentration',
    'composite_hazard_score',
    'tower_height_ft', 'lease_area_sqft', 'parcel_acreage',
    'construction_age', 'site_access_ordinal',
    'permit_approval_difficulty', 'wrluri_index',
    'colocation_required', 'height_restriction_ft',
    'has_building', 'is_ground_tower', 'has_parcel', 'has_zoning_data',
]

Z_NETWORK = [
    'tenant_count', 'primary_technology', 'coverage_critical',
    'capacity_utilization_pct', 'high_traffic',
    'backhaul_type', 'has_fiber_connection',
    'network_substitutability_score', 'search_ring_alternative_count',
    'inter_site_distance_km', 'antenna_count',
    'structure_capacity_remaining', 'site_priority_tier',
]

ALL_FEATURES = X_MARKET + Z_NETWORK

print(f'\nX_MARKET features: {len(X_MARKET)}')
print(f'Z_NETWORK features: {len(Z_NETWORK)}')
print(f'Total features:     {len(ALL_FEATURES)}')

# Verify all features exist in the data
missing = [f for f in ALL_FEATURES if f not in df_raw.columns]
assert len(missing) == 0, f'Missing features: {missing}'
print('All features verified in dataset. ✓')

## 2. Asymmetric Distortion Generator

For each bias level, we create a synthetic `current_monthly_rent` that is **systematically inflated**:

| Site Category | Fraction | Rent Formula |
|:---|:---:|:---|
| **Overpaying** | 70% | `true_FMV × (1 + Uniform(0.05, 2×bias))` |
| **Underpaying** | 15% | `true_FMV × (1 − Uniform(0.05, 0.15))` |
| **At Market** | 15% | `true_FMV × (1 ± Uniform(0, 0.05))` |

This mirrors real portfolios where the majority of leases have accumulated 15-25 years of compounding escalation above market.

In [ ]:
def generate_asymmetric_rents(true_fmv, bias_pct, seed=42):
    """
    Generate asymmetrically distorted rents.
    
    Parameters
    ----------
    true_fmv : array-like
        Ground truth FMV for each site.
    bias_pct : float
        Target average overpayment fraction (e.g. 0.20 = 20%).
    seed : int
        Random seed for reproducibility.
    
    Returns
    -------
    rents : np.ndarray
        Distorted rents.
    categories : np.ndarray
        Site category labels: 'overpaying', 'underpaying', 'at_market'.
    """
    rng = np.random.RandomState(seed)
    n = len(true_fmv)
    fmv = np.asarray(true_fmv, dtype=float)
    
    # Assign categories
    cat_draw = rng.random(n)
    categories = np.where(
        cat_draw < 0.70, 'overpaying',
        np.where(cat_draw < 0.85, 'underpaying', 'at_market')
    )
    
    rents = np.zeros(n)
    
    # Overpaying: rent = FMV * (1 + Uniform(0.05, 2*bias_pct))
    mask_over = categories == 'overpaying'
    n_over = mask_over.sum()
    overpay_factor = 1.0 + rng.uniform(0.05, 2 * bias_pct, n_over)
    rents[mask_over] = fmv[mask_over] * overpay_factor
    
    # Underpaying: rent = FMV * (1 - Uniform(0.05, 0.15))
    mask_under = categories == 'underpaying'
    n_under = mask_under.sum()
    underpay_factor = 1.0 - rng.uniform(0.05, 0.15, n_under)
    rents[mask_under] = fmv[mask_under] * underpay_factor
    
    # At market: rent = FMV * (1 +/- Uniform(0, 0.05))
    mask_at = categories == 'at_market'
    n_at = mask_at.sum()
    at_factor = 1.0 + rng.uniform(-0.05, 0.05, n_at)
    rents[mask_at] = fmv[mask_at] * at_factor
    
    return rents, categories


# ============================================================
# Generate distorted rents for all 4 bias levels
# ============================================================
BIAS_LEVELS = [0.10, 0.20, 0.30, 0.40]
BIAS_LABELS = ['10%', '20%', '30%', '40%']

true_fmv = df_raw['true_fmv'].values

bias_scenarios = {}
for bp, bl in zip(BIAS_LEVELS, BIAS_LABELS):
    rents, cats = generate_asymmetric_rents(true_fmv, bp, seed=42)
    actual_bias = np.mean((rents - true_fmv) / true_fmv)
    bias_scenarios[bl] = {
        'bias_pct': bp,
        'rents': rents,
        'categories': cats,
        'actual_mean_bias': actual_bias,
    }
    print(f'Bias {bl}: actual mean bias = {actual_bias*100:+.1f}%, '
          f'overpaying = {(cats=="overpaying").sum()}, '
          f'underpaying = {(cats=="underpaying").sum()}, '
          f'at_market = {(cats=="at_market").sum()}')

In [ ]:
# ============================================================
# Visualize the distortion for each bias level
# ============================================================
fig, axes = plt.subplots(1, 4, figsize=(20, 4.5))

for ax, bl in zip(axes, BIAS_LABELS):
    sc = bias_scenarios[bl]
    dev = (sc['rents'] - true_fmv) / true_fmv * 100
    ax.hist(dev, bins=60, color='coral', edgecolor='white', alpha=0.8)
    ax.axvline(0, color='black', ls='--', lw=1)
    ax.axvline(np.mean(dev), color='red', ls='-', lw=2,
               label=f'Mean: {np.mean(dev):+.1f}%')
    ax.set_title(f'Bias Level: {bl}', fontweight='bold')
    ax.set_xlabel('Deviation from True FMV (%)')
    ax.legend(fontsize=8)

axes[0].set_ylabel('Site Count')
plt.suptitle('Asymmetric Rent Distortion Distributions', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## 3. Standard Pipeline (HDBSCAN → MAD → XGBoost + QRF)

We run the full pipeline for each bias level. The pipeline stages:

1. **Clustering** — HDBSCAN on standardized `X_MARKET` features to find peer groups
2. **MAD Pseudo-Labeling** — Within each cluster, compute median rent and MAD; sites within 1 MAD of median are "CORE" (pseudo-labeled as at-market)
3. **Supervised Model** — Train XGBoost + QRF on CORE sites using `X_MARKET + Z_NETWORK` → predicts FMV
4. **Classification** — Compare current rent to predicted quantile bands

In [ ]:
def run_pipeline(df, rent_col='current_monthly_rent', x_features=None,
                 z_features=None, min_cluster_size=20, mad_threshold=1.0,
                 label='pipeline'):
    """
    Run the standard FMV estimation pipeline.
    
    Returns dict with: predicted_fmv, q25, q75, labels, cluster_ids, timing.
    """
    if x_features is None:
        x_features = X_MARKET
    if z_features is None:
        z_features = Z_NETWORK
    
    all_feats = x_features + z_features
    rents = df[rent_col].values.copy()
    n = len(df)
    timings = {}
    
    # ---- Stage 1: HDBSCAN Clustering ----
    t0 = time.time()
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df[x_features].values)
    
    clusterer = HDBSCAN(min_cluster_size=min_cluster_size, min_samples=5)
    cluster_ids = clusterer.fit_predict(X_scaled)
    n_clusters = len(set(cluster_ids) - {-1})
    noise_pct = (cluster_ids == -1).mean()
    timings['clustering'] = time.time() - t0
    
    # Assign noise points to nearest cluster
    if (cluster_ids == -1).any():
        from sklearn.neighbors import NearestNeighbors
        valid_mask = cluster_ids != -1
        nn = NearestNeighbors(n_neighbors=1).fit(X_scaled[valid_mask])
        noise_mask = cluster_ids == -1
        _, idx = nn.kneighbors(X_scaled[noise_mask])
        cluster_ids[noise_mask] = cluster_ids[valid_mask][idx.ravel()]
    
    # ---- Stage 2: MAD Pseudo-Labeling ----
    t0 = time.time()
    pseudo_labels = np.full(n, 'EDGE', dtype='U12')
    cluster_medians = np.zeros(n)
    
    for cid in np.unique(cluster_ids):
        mask = cluster_ids == cid
        c_rents = rents[mask]
        med = np.median(c_rents)
        mad = median_abs_deviation(c_rents, scale='normal')
        if mad < 1e-6:
            mad = np.std(c_rents) * 0.6745  # fallback
        
        cluster_medians[mask] = med
        dev = np.abs(c_rents - med) / mad
        
        core = dev <= mad_threshold
        outlier = dev > 2.0
        edge = ~core & ~outlier
        
        idx = np.where(mask)[0]
        pseudo_labels[idx[core]] = 'CORE'
        pseudo_labels[idx[outlier]] = 'OUTLIER'
        pseudo_labels[idx[edge]] = 'EDGE'
    
    core_pct = (pseudo_labels == 'CORE').mean()
    timings['pseudo_labeling'] = time.time() - t0
    
    # ---- Stage 3: Supervised Model on CORE sites ----
    t0 = time.time()
    core_mask = pseudo_labels == 'CORE'
    X_train = df.loc[core_mask, all_feats].values
    y_train = rents[core_mask]  # CORE rents used as pseudo-FMV targets
    X_all = df[all_feats].values
    
    # XGBoost point estimate
    xgb = XGBRegressor(
        n_estimators=500, learning_rate=0.05, max_depth=6,
        subsample=0.8, colsample_bytree=0.8,
        min_child_weight=5, reg_alpha=0.1, reg_lambda=1.0,
        random_state=42, n_jobs=-1
    )
    xgb.fit(X_train, y_train)
    pred_xgb = xgb.predict(X_all)
    
    # QRF for quantile bands
    if HAS_QRF:
        qrf = RandomForestQuantileRegressor(
            n_estimators=500, min_samples_leaf=5,
            random_state=42, n_jobs=-1
        )
        qrf.fit(X_train, y_train)
        pred_q25 = qrf.predict(X_all, quantiles=0.25)
        pred_q50 = qrf.predict(X_all, quantiles=0.50)
        pred_q75 = qrf.predict(X_all, quantiles=0.75)
    else:
        rf = RandomForestRegressor(
            n_estimators=500, min_samples_leaf=5,
            random_state=42, n_jobs=-1
        )
        rf.fit(X_train, y_train)
        pred_q50 = rf.predict(X_all)
        # Approximate quantiles with +/- 15% band
        pred_q25 = pred_q50 * 0.85
        pred_q75 = pred_q50 * 1.15
    
    # Meta-learner: Ridge blend of XGB + QRF median
    meta_X = np.column_stack([pred_xgb, pred_q50])
    meta_X_train = meta_X[core_mask]
    meta_y_train = y_train
    ridge = Ridge(alpha=1.0)
    ridge.fit(meta_X_train, meta_y_train)
    predicted_fmv = ridge.predict(meta_X)
    predicted_fmv = np.maximum(predicted_fmv, 100)  # floor
    
    timings['modeling'] = time.time() - t0
    
    # ---- Stage 4: Classification ----
    pred_class = np.where(
        rents > pred_q75, 'OVERPAYING',
        np.where(rents < pred_q25, 'UNDERPAYING', 'AT_MARKET')
    )
    
    return {
        'predicted_fmv': predicted_fmv,
        'pred_q25': pred_q25,
        'pred_q50': pred_q50,
        'pred_q75': pred_q75,
        'pred_class': pred_class,
        'pseudo_labels': pseudo_labels,
        'cluster_ids': cluster_ids,
        'n_clusters': n_clusters,
        'noise_pct': noise_pct,
        'core_pct': core_pct,
        'timings': timings,
    }


print('Pipeline function defined.')

In [ ]:
# ============================================================
# Run pipeline for each bias level
# ============================================================
results = {}

for bl in BIAS_LABELS:
    print(f'\n{"="*60}')
    print(f'Running pipeline for bias level = {bl}')
    print(f'{"="*60}')
    
    sc = bias_scenarios[bl]
    
    # Create working copy with distorted rents
    df_work = df_raw.copy()
    df_work['current_monthly_rent'] = sc['rents']
    
    # Run the standard pipeline
    res = run_pipeline(df_work, rent_col='current_monthly_rent')
    res['distorted_rents'] = sc['rents']
    res['true_categories'] = sc['categories']
    results[bl] = res
    
    # Quick metrics
    pred = res['predicted_fmv']
    bias = np.mean(pred - true_fmv)
    mape = np.mean(np.abs(pred - true_fmv) / true_fmv) * 100
    mae = mean_absolute_error(true_fmv, pred)
    bias_dir = 'POSITIVE (inflated)' if bias > 0 else 'NEGATIVE (deflated)'
    
    print(f'  Clusters: {res["n_clusters"]}, Noise: {res["noise_pct"]*100:.1f}%, '
          f'Core: {res["core_pct"]*100:.1f}%')
    print(f'  Prediction Bias: ${bias:+,.0f}/mo ({bias_dir})')
    print(f'  MAPE vs true FMV: {mape:.1f}%')
    print(f'  MAE vs true FMV: ${mae:,.0f}/mo')
    print(f'  Total time: {sum(res["timings"].values()):.1f}s')

## 4. Measure Bias Leakage into Predicted FMV

The critical question: **how much of the injected overpayment leaked into predicted FMV?**

We compute:
- **Bias** = mean(predicted_FMV − true_FMV) — positive = model inherited the overpayment
- **Bias leakage rate** = model_bias / injected_bias — 100% means no detection, 0% means perfect detection
- **MAPE** vs true FMV
- **MAE** vs true FMV

In [ ]:
# ============================================================
# Bias Measurement Table
# ============================================================
bias_rows = []
for bl in BIAS_LABELS:
    sc = bias_scenarios[bl]
    res = results[bl]
    pred = res['predicted_fmv']
    
    # Injected bias (in the input rents)
    injected_bias_dollar = np.mean(sc['rents'] - true_fmv)
    injected_bias_pct = np.mean((sc['rents'] - true_fmv) / true_fmv) * 100
    
    # Model bias (in the predictions)
    model_bias_dollar = np.mean(pred - true_fmv)
    model_bias_pct = np.mean((pred - true_fmv) / true_fmv) * 100
    
    # Leakage rate
    leakage = (model_bias_dollar / injected_bias_dollar * 100) if injected_bias_dollar > 0 else 0
    
    # Detection: how much of the overpayment does the model flag?
    detection = 100 - leakage
    
    # Error metrics
    mape = np.mean(np.abs(pred - true_fmv) / true_fmv) * 100
    mae = mean_absolute_error(true_fmv, pred)
    r2 = r2_score(true_fmv, pred)
    
    bias_rows.append({
        'Bias Level': bl,
        'Injected Bias ($)': f'${injected_bias_dollar:+,.0f}',
        'Injected Bias (%)': f'{injected_bias_pct:+.1f}%',
        'Model Bias ($)': f'${model_bias_dollar:+,.0f}',
        'Model Bias (%)': f'{model_bias_pct:+.1f}%',
        'Leakage Rate': f'{leakage:.1f}%',
        'Detection Rate': f'{detection:.1f}%',
        'MAPE': f'{mape:.1f}%',
        'MAE ($)': f'${mae:,.0f}',
        'R²': f'{r2:.3f}',
    })

bias_df = pd.DataFrame(bias_rows)
print('\n' + '='*80)
print('BIAS LEAKAGE ANALYSIS')
print('='*80)
print(bias_df.to_string(index=False))
print('\nInterpretation:')
print('  - Leakage Rate: % of injected bias that leaked into predicted FMV')
print('  - Detection Rate: % of injected bias the model effectively removes')
print('  - A good pipeline has LOW leakage and HIGH detection.')

In [ ]:
# ============================================================
# Plot: Model Bias vs True (Injected) Bias
# ============================================================
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# --- Plot 1: Model Bias vs Injected Bias ---
injected = [np.mean((bias_scenarios[bl]['rents'] - true_fmv) / true_fmv) * 100
            for bl in BIAS_LABELS]
model_bias_vals = [np.mean((results[bl]['predicted_fmv'] - true_fmv) / true_fmv) * 100
                   for bl in BIAS_LABELS]

ax = axes[0]
ax.plot(injected, model_bias_vals, 'ro-', ms=10, lw=2, label='Model bias')
ax.plot([0, max(injected)*1.1], [0, max(injected)*1.1], 'k--', lw=1,
        alpha=0.5, label='No detection (100% leakage)')
ax.plot([0, max(injected)*1.1], [0, 0], 'g--', lw=1,
        alpha=0.5, label='Perfect detection (0% leakage)')
for i, bl in enumerate(BIAS_LABELS):
    ax.annotate(bl, (injected[i], model_bias_vals[i]),
                textcoords='offset points', xytext=(10, 5), fontsize=9)
ax.set_xlabel('Injected Bias (% of true FMV)')
ax.set_ylabel('Model Bias (% of true FMV)')
ax.set_title('Model Bias vs Injected Bias', fontweight='bold')
ax.legend(fontsize=8)

# --- Plot 2: MAPE at each bias level ---
mapes = [np.mean(np.abs(results[bl]['predicted_fmv'] - true_fmv) / true_fmv) * 100
         for bl in BIAS_LABELS]

ax = axes[1]
bars = ax.bar(BIAS_LABELS, mapes, color=['#2ecc71', '#f39c12', '#e67e22', '#e74c3c'],
              edgecolor='white', width=0.6)
for bar, val in zip(bars, mapes):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3,
            f'{val:.1f}%', ha='center', fontsize=10, fontweight='bold')
ax.set_xlabel('Bias Level')
ax.set_ylabel('MAPE vs True FMV (%)')
ax.set_title('Prediction Error by Bias Level', fontweight='bold')
ax.axhline(15, color='green', ls='--', alpha=0.4, label='15% target')
ax.legend(fontsize=8)

# --- Plot 3: Leakage rate ---
leakages = []
for bl in BIAS_LABELS:
    inj = np.mean(bias_scenarios[bl]['rents'] - true_fmv)
    mod = np.mean(results[bl]['predicted_fmv'] - true_fmv)
    leakages.append(mod / inj * 100 if inj > 0 else 0)

ax = axes[2]
ax.bar(BIAS_LABELS, leakages, color=['#3498db', '#3498db', '#e74c3c', '#e74c3c'],
       edgecolor='white', width=0.6)
for i, (bl, lk) in enumerate(zip(BIAS_LABELS, leakages)):
    ax.text(i, lk + 1, f'{lk:.0f}%', ha='center', fontsize=10, fontweight='bold')
ax.set_xlabel('Bias Level')
ax.set_ylabel('Bias Leakage Rate (%)')
ax.set_title('How Much Bias Leaks Through?', fontweight='bold')
ax.axhline(50, color='orange', ls='--', alpha=0.5, label='50% threshold')
ax.legend(fontsize=8)

plt.suptitle('Simulation 03 -- Asymmetric Bias Impact', fontsize=14, y=1.03)
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# Scatter: Predicted FMV vs True FMV at WORST bias level (40%)
# ============================================================
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

worst = results['40%']
pred_40 = worst['predicted_fmv']

# Left: colored by true category
ax = axes[0]
cat_colors = {'overpaying': '#e74c3c', 'underpaying': '#3498db', 'at_market': '#2ecc71'}
cats_40 = worst['true_categories']
for cat, color in cat_colors.items():
    mask = cats_40 == cat
    ax.scatter(true_fmv[mask], pred_40[mask], alpha=0.3, s=8,
              c=color, label=f'{cat} ({mask.sum()})')
lims = [0, max(true_fmv.max(), pred_40.max()) * 1.05]
ax.plot(lims, lims, 'k--', lw=1, alpha=0.5)
ax.set_xlabel('True FMV ($/mo)')
ax.set_ylabel('Predicted FMV ($/mo)')
ax.set_title('Predicted vs True FMV @ 40% Bias\n(colored by true category)', fontweight='bold')
ax.legend(fontsize=8, loc='upper left')

# Right: residual distribution
ax = axes[1]
residual = (pred_40 - true_fmv) / true_fmv * 100
ax.hist(residual, bins=60, color='coral', edgecolor='white', alpha=0.8)
ax.axvline(0, color='black', ls='--', lw=1.5)
ax.axvline(np.mean(residual), color='red', ls='-', lw=2,
           label=f'Mean: {np.mean(residual):+.1f}%')
ax.axvline(np.median(residual), color='blue', ls='-', lw=2,
           label=f'Median: {np.median(residual):+.1f}%')
ax.set_xlabel('Prediction Error (% of true FMV)')
ax.set_ylabel('Count')
ax.set_title('Residual Distribution @ 40% Bias', fontweight='bold')
ax.legend(fontsize=9)

plt.tight_layout()
plt.show()

## 5. Defense Layer Detection Ability

The pipeline includes defense layers designed to catch systematic bias. We test three:

1. **External Benchmark Check** — Does the predicted FMV fall within published market ranges?
2. **Land Value Correlation** — Is predicted FMV correlated with underlying land values? (Low correlation = suspicious)
3. **Isolation Forest** — Anomaly detection on features + rent; does it agree with model classification?

In [ ]:
# ============================================================
# Defense Layer 1: External Benchmark Check
# ============================================================
# Define benchmark FMV ranges by market type
# These approximate published tower lease rate surveys
BENCHMARK_RANGES = {
    'urban':    (1200, 2500),   # urban ground towers
    'suburban': (800, 1500),    # suburban ground towers
    'rural':    (500, 1000),    # rural ground towers
    'rooftop':  (1500, 3500),   # rooftop leases (all areas)
}

def benchmark_check(df, predicted_fmv, threshold=0.20):
    """
    Check what % of predicted FMVs land ABOVE benchmark ceilings.
    If >threshold are above, flag as bias detected.
    """
    n = len(df)
    above_ceiling = np.zeros(n, dtype=bool)
    
    # Assign each site to a benchmark category
    urc = df['urban_rural_class'].values
    is_rooftop = (df['property_type'].values == 'rooftop') if 'property_type' in df.columns else (df['has_building'].values == 1)
    
    for i in range(n):
        if is_rooftop[i] if hasattr(is_rooftop, '__getitem__') else False:
            ceiling = BENCHMARK_RANGES['rooftop'][1]
        elif urc[i] >= 4:
            ceiling = BENCHMARK_RANGES['urban'][1]
        elif urc[i] >= 3:
            ceiling = BENCHMARK_RANGES['suburban'][1]
        else:
            ceiling = BENCHMARK_RANGES['rural'][1]
        
        above_ceiling[i] = predicted_fmv[i] > ceiling
    
    pct_above = above_ceiling.mean()
    detected = pct_above > threshold
    
    return {
        'pct_above_ceiling': pct_above,
        'bias_detected': detected,
        'threshold': threshold,
    }


# ============================================================
# Defense Layer 2: Land Value Correlation
# ============================================================
def land_value_correlation(df, predicted_fmv):
    """
    Compute Pearson r between predicted FMV and underlying_land_value_psf.
    A well-calibrated model should show moderate positive correlation.
    Very low correlation suggests the model is driven by bias rather than fundamentals.
    """
    land_val = df['underlying_land_value_psf'].values
    r, pval = pearsonr(predicted_fmv, land_val)
    return {'pearson_r': r, 'p_value': pval}


# ============================================================
# Defense Layer 3: Isolation Forest Agreement
# ============================================================
def isolation_forest_check(df, rents, pred_class, all_features=None):
    """
    Train Isolation Forest on features + rent, check Cohen's Kappa
    with the model's classification.
    """
    if all_features is None:
        all_features = ALL_FEATURES
    
    X_if = np.column_stack([df[all_features].values, rents.reshape(-1, 1)])
    scaler = StandardScaler()
    X_if_scaled = scaler.fit_transform(X_if)
    
    iso = IsolationForest(n_estimators=500, contamination='auto',
                          random_state=42, n_jobs=-1)
    iso_labels = iso.fit_predict(X_if_scaled)  # 1 = inlier, -1 = outlier
    
    # Convert to binary: outlier → OVERPAYING, inlier → AT_MARKET/UNDERPAYING
    iso_binary = np.where(iso_labels == -1, 1, 0)  # 1 = outlier
    model_binary = np.where(pred_class == 'OVERPAYING', 1, 0)
    
    kappa = cohen_kappa_score(iso_binary, model_binary)
    return {
        'kappa': kappa,
        'iso_outlier_pct': (iso_labels == -1).mean(),
        'model_overpay_pct': (pred_class == 'OVERPAYING').mean(),
    }


print('Defense layer functions defined.')

In [ ]:
# ============================================================
# Run all defense layers for each bias level
# ============================================================
defense_results = {}

for bl in BIAS_LABELS:
    print(f'\n--- Defense layers for bias = {bl} ---')
    sc = bias_scenarios[bl]
    res = results[bl]
    
    df_work = df_raw.copy()
    df_work['current_monthly_rent'] = sc['rents']
    
    # Benchmark check
    bench = benchmark_check(df_work, res['predicted_fmv'])
    print(f'  Benchmark: {bench["pct_above_ceiling"]*100:.1f}% above ceiling '
          f'-> {"DETECTED" if bench["bias_detected"] else "NOT detected"}')
    
    # Land value correlation
    lv = land_value_correlation(df_work, res['predicted_fmv'])
    print(f'  Land value Pearson r: {lv["pearson_r"]:.3f} (p={lv["p_value"]:.2e})')
    
    # Isolation Forest
    iso = isolation_forest_check(df_work, sc['rents'], res['pred_class'])
    print(f'  Isolation Forest Kappa: {iso["kappa"]:.3f} '
          f'(IF outlier: {iso["iso_outlier_pct"]*100:.1f}%, '
          f'Model overpay: {iso["model_overpay_pct"]*100:.1f}%)')
    
    defense_results[bl] = {
        'benchmark': bench,
        'land_corr': lv,
        'isolation_forest': iso,
    }

## 6. Fallback Corrections

When bias is detected, we need **fallback strategies** that reduce it. We test two:

### Fallback A: Newer-Lease Anchoring
Use only sites with `lease_age_years < 5` as pseudo-labels. Rationale: newer leases haven't accumulated many years of escalation, so they're closer to current market.

### Fallback B: Cluster Q25 Anchoring  
Instead of using cluster **median** rent as the pseudo-FMV anchor, use the **25th percentile**. Rationale: if most rents are inflated, Q25 is closer to true market than median.

In [ ]:
def run_pipeline_fallback_a(df, rent_col='current_monthly_rent',
                            max_lease_age=5, x_features=None, z_features=None):
    """
    Fallback A: Use only newer leases (lease_age < max_lease_age)
    as pseudo-label anchors. Older leases still get predicted but
    are NOT used for training.
    """
    if x_features is None:
        x_features = X_MARKET
    if z_features is None:
        z_features = Z_NETWORK
    
    all_feats = x_features + z_features
    rents = df[rent_col].values.copy()
    n = len(df)
    
    # Stage 1: Clustering (same as standard)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df[x_features].values)
    clusterer = HDBSCAN(min_cluster_size=20, min_samples=5)
    cluster_ids = clusterer.fit_predict(X_scaled)
    
    # Assign noise to nearest cluster
    if (cluster_ids == -1).any():
        from sklearn.neighbors import NearestNeighbors
        valid_mask = cluster_ids != -1
        nn = NearestNeighbors(n_neighbors=1).fit(X_scaled[valid_mask])
        noise_mask = cluster_ids == -1
        _, idx = nn.kneighbors(X_scaled[noise_mask])
        cluster_ids[noise_mask] = cluster_ids[valid_mask][idx.ravel()]
    
    # Stage 2: MAD -- but restrict to newer leases
    newer_mask = df['lease_age_years'].values < max_lease_age
    pseudo_labels = np.full(n, 'EDGE', dtype='U12')
    
    for cid in np.unique(cluster_ids):
        mask = (cluster_ids == cid) & newer_mask
        if mask.sum() < 3:
            # Not enough newer leases in this cluster; use all sites
            mask = cluster_ids == cid
        
        c_rents = rents[mask]
        med = np.median(c_rents)
        mad = median_abs_deviation(c_rents, scale='normal')
        if mad < 1e-6:
            mad = np.std(c_rents) * 0.6745
        
        # Apply MAD labeling to ALL sites in the cluster
        all_in_cluster = cluster_ids == cid
        dev = np.abs(rents[all_in_cluster] - med) / mad
        core = dev <= 1.0
        idx = np.where(all_in_cluster)[0]
        pseudo_labels[idx[core]] = 'CORE'
    
    # Force only newer CORE sites into training
    core_mask = (pseudo_labels == 'CORE') & newer_mask
    if core_mask.sum() < 50:
        # Fallback: if too few, use all CORE sites
        core_mask = pseudo_labels == 'CORE'
    
    # Stage 3: Model
    X_all = df[all_feats].values
    X_train = X_all[core_mask]
    y_train = rents[core_mask]
    
    xgb = XGBRegressor(
        n_estimators=500, learning_rate=0.05, max_depth=6,
        subsample=0.8, colsample_bytree=0.8,
        min_child_weight=5, reg_alpha=0.1, reg_lambda=1.0,
        random_state=42, n_jobs=-1
    )
    xgb.fit(X_train, y_train)
    predicted_fmv = np.maximum(xgb.predict(X_all), 100)
    
    return predicted_fmv, core_mask.sum()


def run_pipeline_fallback_b(df, rent_col='current_monthly_rent',
                            x_features=None, z_features=None):
    """
    Fallback B: Use cluster Q25 (instead of median) as pseudo-FMV anchor.
    This downward-adjusts the training targets to counteract inflation.
    """
    if x_features is None:
        x_features = X_MARKET
    if z_features is None:
        z_features = Z_NETWORK
    
    all_feats = x_features + z_features
    rents = df[rent_col].values.copy()
    n = len(df)
    
    # Stage 1: Clustering
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df[x_features].values)
    clusterer = HDBSCAN(min_cluster_size=20, min_samples=5)
    cluster_ids = clusterer.fit_predict(X_scaled)
    
    if (cluster_ids == -1).any():
        from sklearn.neighbors import NearestNeighbors
        valid_mask = cluster_ids != -1
        nn = NearestNeighbors(n_neighbors=1).fit(X_scaled[valid_mask])
        noise_mask = cluster_ids == -1
        _, idx = nn.kneighbors(X_scaled[noise_mask])
        cluster_ids[noise_mask] = cluster_ids[valid_mask][idx.ravel()]
    
    # Stage 2: MAD -- anchor on Q25 instead of median
    pseudo_labels = np.full(n, 'EDGE', dtype='U12')
    adjusted_targets = rents.copy()
    
    for cid in np.unique(cluster_ids):
        mask = cluster_ids == cid
        c_rents = rents[mask]
        q25 = np.percentile(c_rents, 25)
        mad = median_abs_deviation(c_rents, scale='normal')
        if mad < 1e-6:
            mad = np.std(c_rents) * 0.6745
        
        dev = np.abs(c_rents - q25) / mad
        core = dev <= 1.5  # slightly wider band around Q25
        idx = np.where(mask)[0]
        pseudo_labels[idx[core]] = 'CORE'
        # Adjust targets: use Q25-centered value
        adjusted_targets[mask] = rents[mask] - (np.median(c_rents) - q25)
    
    core_mask = pseudo_labels == 'CORE'
    
    # Stage 3: Model on adjusted targets
    X_all = df[all_feats].values
    X_train = X_all[core_mask]
    y_train = adjusted_targets[core_mask]
    
    xgb = XGBRegressor(
        n_estimators=500, learning_rate=0.05, max_depth=6,
        subsample=0.8, colsample_bytree=0.8,
        min_child_weight=5, reg_alpha=0.1, reg_lambda=1.0,
        random_state=42, n_jobs=-1
    )
    xgb.fit(X_train, y_train)
    predicted_fmv = np.maximum(xgb.predict(X_all), 100)
    
    return predicted_fmv, core_mask.sum()


print('Fallback pipeline functions defined.')

In [ ]:
# ============================================================
# Run fallback corrections for each bias level
# ============================================================
fallback_results = {}

for bl in BIAS_LABELS:
    print(f'\n--- Fallback corrections for bias = {bl} ---')
    sc = bias_scenarios[bl]
    res = results[bl]
    
    df_work = df_raw.copy()
    df_work['current_monthly_rent'] = sc['rents']
    
    # Standard pipeline bias
    std_bias = np.mean(res['predicted_fmv'] - true_fmv)
    std_mape = np.mean(np.abs(res['predicted_fmv'] - true_fmv) / true_fmv) * 100
    
    # Fallback A: Newer-lease anchoring
    pred_a, n_train_a = run_pipeline_fallback_a(df_work)
    bias_a = np.mean(pred_a - true_fmv)
    mape_a = np.mean(np.abs(pred_a - true_fmv) / true_fmv) * 100
    bias_reduction_a = (1 - abs(bias_a) / abs(std_bias)) * 100 if abs(std_bias) > 0 else 0
    
    # Fallback B: Q25 anchoring
    pred_b, n_train_b = run_pipeline_fallback_b(df_work)
    bias_b = np.mean(pred_b - true_fmv)
    mape_b = np.mean(np.abs(pred_b - true_fmv) / true_fmv) * 100
    bias_reduction_b = (1 - abs(bias_b) / abs(std_bias)) * 100 if abs(std_bias) > 0 else 0
    
    print(f'  Standard:    bias=${std_bias:+,.0f}, MAPE={std_mape:.1f}%')
    print(f'  Fallback A:  bias=${bias_a:+,.0f}, MAPE={mape_a:.1f}% '
          f'(bias reduction: {bias_reduction_a:+.0f}%, train_n={n_train_a})')
    print(f'  Fallback B:  bias=${bias_b:+,.0f}, MAPE={mape_b:.1f}% '
          f'(bias reduction: {bias_reduction_b:+.0f}%, train_n={n_train_b})')
    
    fallback_results[bl] = {
        'standard': {'bias': std_bias, 'mape': std_mape},
        'fallback_a': {'bias': bias_a, 'mape': mape_a, 'pred': pred_a,
                       'bias_reduction': bias_reduction_a, 'n_train': n_train_a},
        'fallback_b': {'bias': bias_b, 'mape': mape_b, 'pred': pred_b,
                       'bias_reduction': bias_reduction_b, 'n_train': n_train_b},
    }

In [ ]:
# ============================================================
# Plot: Before/After Fallback Correction Comparison
# ============================================================
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# --- Plot 1: Bias reduction ---
ax = axes[0]
x_pos = np.arange(len(BIAS_LABELS))
width = 0.25

std_biases = [fallback_results[bl]['standard']['bias'] for bl in BIAS_LABELS]
fb_a_biases = [fallback_results[bl]['fallback_a']['bias'] for bl in BIAS_LABELS]
fb_b_biases = [fallback_results[bl]['fallback_b']['bias'] for bl in BIAS_LABELS]

bars1 = ax.bar(x_pos - width, std_biases, width, label='Standard Pipeline',
               color='#e74c3c', edgecolor='white')
bars2 = ax.bar(x_pos, fb_a_biases, width, label='Fallback A (Newer Lease)',
               color='#f39c12', edgecolor='white')
bars3 = ax.bar(x_pos + width, fb_b_biases, width, label='Fallback B (Q25)',
               color='#2ecc71', edgecolor='white')

ax.set_xticks(x_pos)
ax.set_xticklabels(BIAS_LABELS)
ax.set_xlabel('Bias Level')
ax.set_ylabel('Residual Bias ($/mo)')
ax.set_title('Residual Bias: Standard vs Fallbacks', fontweight='bold')
ax.axhline(0, color='black', ls='-', lw=0.5)
ax.legend(fontsize=9)

# --- Plot 2: MAPE comparison ---
ax = axes[1]
std_mapes = [fallback_results[bl]['standard']['mape'] for bl in BIAS_LABELS]
fb_a_mapes = [fallback_results[bl]['fallback_a']['mape'] for bl in BIAS_LABELS]
fb_b_mapes = [fallback_results[bl]['fallback_b']['mape'] for bl in BIAS_LABELS]

ax.plot(BIAS_LABELS, std_mapes, 'rs-', ms=10, lw=2, label='Standard Pipeline')
ax.plot(BIAS_LABELS, fb_a_mapes, 'o-', color='#f39c12', ms=10, lw=2,
        label='Fallback A (Newer Lease)')
ax.plot(BIAS_LABELS, fb_b_mapes, 'g^-', ms=10, lw=2, label='Fallback B (Q25)')
ax.axhline(15, color='green', ls='--', alpha=0.4, label='15% target')
ax.set_xlabel('Bias Level')
ax.set_ylabel('MAPE vs True FMV (%)')
ax.set_title('MAPE: Standard vs Fallbacks', fontweight='bold')
ax.legend(fontsize=9)

plt.suptitle('Simulation 03 -- Fallback Correction Effectiveness', fontsize=14, y=1.03)
plt.tight_layout()
plt.show()

## 7. Bias Detection Scorecard

This is the master table that summarizes all findings: for each bias level, which defense layers fired, and how much bias remained after each fallback.

In [ ]:
# ============================================================
# Build Comprehensive Bias Detection Scorecard
# ============================================================
scorecard_rows = []

for bl in BIAS_LABELS:
    sc = bias_scenarios[bl]
    res = results[bl]
    dr = defense_results[bl]
    fb = fallback_results[bl]
    
    # Injected bias
    injected_pct = np.mean((sc['rents'] - true_fmv) / true_fmv) * 100
    
    # Model residual bias
    model_bias_pct = np.mean((res['predicted_fmv'] - true_fmv) / true_fmv) * 100
    
    # Defense detections
    bench_detected = dr['benchmark']['bias_detected']
    bench_pct = dr['benchmark']['pct_above_ceiling'] * 100
    land_r = dr['land_corr']['pearson_r']
    land_detected = land_r < 0.40  # suspiciously low
    iso_kappa = dr['isolation_forest']['kappa']
    iso_detected = iso_kappa > 0.20  # meaningful agreement
    
    # Fallback residual biases
    fb_a_bias = np.mean((fb['fallback_a']['pred'] - true_fmv) / true_fmv) * 100
    fb_b_bias = np.mean((fb['fallback_b']['pred'] - true_fmv) / true_fmv) * 100
    
    scorecard_rows.append({
        'Bias Level': bl,
        'Injected Bias (%)': f'{injected_pct:+.1f}%',
        'Model Bias (%)': f'{model_bias_pct:+.1f}%',
        'Benchmark': f'YES ({bench_pct:.0f}%)' if bench_detected else f'no ({bench_pct:.0f}%)',
        'IF Kappa': f'YES ({iso_kappa:.2f})' if iso_detected else f'no ({iso_kappa:.2f})',
        'Land Corr': f'WARN (r={land_r:.2f})' if land_detected else f'ok (r={land_r:.2f})',
        'FB-A Bias (%)': f'{fb_a_bias:+.1f}%',
        'FB-B Bias (%)': f'{fb_b_bias:+.1f}%',
        'Best Fallback': 'A' if abs(fb_a_bias) < abs(fb_b_bias) else 'B',
    })

scorecard_df = pd.DataFrame(scorecard_rows)

print('\n' + '='*100)
print('BIAS DETECTION SCORECARD')
print('='*100)
print(scorecard_df.to_string(index=False))
print('\n--- Legend ---')
print('  Benchmark: YES/no = bias detected/not detected by external benchmark check (>20% above ceiling)')
print('  IF Kappa:  YES/no = Isolation Forest Kappa > 0.20 indicates meaningful agreement with model')
print('  Land Corr: WARN/ok = Pearson r < 0.40 with land value suggests model driven by bias not fundamentals')
print('  FB-A: Newer-lease anchoring residual bias')
print('  FB-B: Q25 anchoring residual bias')

## 8. Final Visualization: Full Story

Three key plots that tell the complete story of this simulation.

In [ ]:
# ============================================================
# Final comprehensive visualization
# ============================================================
fig, axes = plt.subplots(2, 3, figsize=(20, 12))

# --- (0,0) Model bias vs true bias (4 points) ---
ax = axes[0, 0]
injected_vals = [np.mean((bias_scenarios[bl]['rents'] - true_fmv) / true_fmv) * 100
                 for bl in BIAS_LABELS]
model_vals = [np.mean((results[bl]['predicted_fmv'] - true_fmv) / true_fmv) * 100
              for bl in BIAS_LABELS]
ax.plot(injected_vals, model_vals, 'ro-', ms=12, lw=2.5, zorder=5)
ax.plot([0, max(injected_vals)*1.15], [0, max(injected_vals)*1.15],
        'k--', lw=1, alpha=0.4, label='100% leakage')
ax.plot([0, max(injected_vals)*1.15], [0, 0],
        'g--', lw=1, alpha=0.4, label='0% leakage')
ax.fill_between([0, max(injected_vals)*1.15], [0, 0],
                [0, max(injected_vals)*1.15], alpha=0.05, color='red')
for i, bl in enumerate(BIAS_LABELS):
    ax.annotate(bl, (injected_vals[i], model_vals[i]),
                textcoords='offset points', xytext=(10, 8), fontsize=10,
                fontweight='bold')
ax.set_xlabel('Injected Bias (%)', fontsize=11)
ax.set_ylabel('Model Bias (%)', fontsize=11)
ax.set_title('Bias Leakage: Model vs Injected', fontweight='bold')
ax.legend(fontsize=9)

# --- (0,1) Scatter: pred vs true at 40% bias ---
ax = axes[0, 1]
pred_40 = results['40%']['predicted_fmv']
ax.scatter(true_fmv, pred_40, alpha=0.15, s=6, c='#e74c3c')
lims = [0, max(true_fmv.max(), pred_40.max()) * 1.05]
ax.plot(lims, lims, 'k--', lw=1.5, alpha=0.6, label='Perfect prediction')
ax.set_xlabel('True FMV ($/mo)', fontsize=11)
ax.set_ylabel('Predicted FMV ($/mo)', fontsize=11)
ax.set_title('Predicted vs True FMV @ 40% Bias', fontweight='bold')
ax.legend(fontsize=9)

# --- (0,2) Defense layer summary heatmap ---
ax = axes[0, 2]
defense_matrix = np.zeros((len(BIAS_LABELS), 3))
for i, bl in enumerate(BIAS_LABELS):
    dr = defense_results[bl]
    defense_matrix[i, 0] = 1 if dr['benchmark']['bias_detected'] else 0
    defense_matrix[i, 1] = 1 if dr['isolation_forest']['kappa'] > 0.20 else 0
    defense_matrix[i, 2] = 1 if dr['land_corr']['pearson_r'] < 0.40 else 0

sns.heatmap(defense_matrix, annot=True, fmt='.0f', cmap='RdYlGn_r',
            xticklabels=['Benchmark', 'Iso Forest', 'Land Corr'],
            yticklabels=BIAS_LABELS, ax=ax, cbar=False,
            linewidths=2, linecolor='white', vmin=0, vmax=1)
ax.set_xlabel('Defense Layer')
ax.set_ylabel('Bias Level')
ax.set_title('Defense Detection (1=Fired, 0=Missed)', fontweight='bold')

# --- (1,0) Before/after fallback (bias) ---
ax = axes[1, 0]
x_pos = np.arange(len(BIAS_LABELS))
width = 0.22
std_b = [fallback_results[bl]['standard']['bias'] for bl in BIAS_LABELS]
fba_b = [fallback_results[bl]['fallback_a']['bias'] for bl in BIAS_LABELS]
fbb_b = [fallback_results[bl]['fallback_b']['bias'] for bl in BIAS_LABELS]

ax.bar(x_pos - width, std_b, width, label='Standard', color='#e74c3c', edgecolor='white')
ax.bar(x_pos, fba_b, width, label='FB-A (Newer)', color='#f39c12', edgecolor='white')
ax.bar(x_pos + width, fbb_b, width, label='FB-B (Q25)', color='#2ecc71', edgecolor='white')
ax.set_xticks(x_pos)
ax.set_xticklabels(BIAS_LABELS)
ax.set_xlabel('Bias Level')
ax.set_ylabel('Residual Bias ($/mo)')
ax.set_title('Fallback Correction: Residual Bias', fontweight='bold')
ax.axhline(0, color='black', ls='-', lw=0.5)
ax.legend(fontsize=9)

# --- (1,1) Before/after fallback (MAPE) ---
ax = axes[1, 1]
std_m = [fallback_results[bl]['standard']['mape'] for bl in BIAS_LABELS]
fba_m = [fallback_results[bl]['fallback_a']['mape'] for bl in BIAS_LABELS]
fbb_m = [fallback_results[bl]['fallback_b']['mape'] for bl in BIAS_LABELS]

ax.plot(BIAS_LABELS, std_m, 'rs-', ms=10, lw=2, label='Standard')
ax.plot(BIAS_LABELS, fba_m, 'o-', color='#f39c12', ms=10, lw=2, label='FB-A (Newer)')
ax.plot(BIAS_LABELS, fbb_m, 'g^-', ms=10, lw=2, label='FB-B (Q25)')
ax.axhline(15, color='green', ls='--', alpha=0.4, label='15% target')
ax.set_xlabel('Bias Level')
ax.set_ylabel('MAPE (%)')
ax.set_title('Fallback Correction: MAPE', fontweight='bold')
ax.legend(fontsize=9)

# --- (1,2) Scatter at 40% bias: standard vs best fallback ---
ax = axes[1, 2]
best_fb_key = 'fallback_a' if abs(fallback_results['40%']['fallback_a']['bias']) < \
    abs(fallback_results['40%']['fallback_b']['bias']) else 'fallback_b'
best_fb_label = 'Newer-Lease' if best_fb_key == 'fallback_a' else 'Q25'
pred_fb = fallback_results['40%'][best_fb_key]['pred']

ax.scatter(true_fmv, pred_40, alpha=0.15, s=6, c='#e74c3c', label='Standard')
ax.scatter(true_fmv, pred_fb, alpha=0.15, s=6, c='#2ecc71', label=f'Best Fallback ({best_fb_label})')
ax.plot(lims, lims, 'k--', lw=1.5, alpha=0.6)
ax.set_xlabel('True FMV ($/mo)', fontsize=11)
ax.set_ylabel('Predicted FMV ($/mo)', fontsize=11)
ax.set_title(f'Standard vs {best_fb_label} Fallback @ 40% Bias', fontweight='bold')
ax.legend(fontsize=9, loc='upper left')

plt.suptitle('Simulation 03 -- Asymmetric Bias Stress Test: Complete Results',
             fontsize=15, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 9. Conclusions & Recommendations

### Key Findings

In [ ]:
# ============================================================
# Print final conclusions
# ============================================================
print('='*80)
print('SIMULATION 03 -- CONCLUSIONS')
print('='*80)

# Leakage summary
print('\n1. BIAS LEAKAGE:')
for bl in BIAS_LABELS:
    sc = bias_scenarios[bl]
    res = results[bl]
    inj = np.mean(sc['rents'] - true_fmv)
    mod = np.mean(res['predicted_fmv'] - true_fmv)
    leak = mod / inj * 100 if inj > 0 else 0
    print(f'   Bias {bl}: {leak:.0f}% of injected bias leaked through')

# Defense summary
print('\n2. DEFENSE LAYER EFFECTIVENESS:')
for bl in BIAS_LABELS:
    dr = defense_results[bl]
    detections = []
    if dr['benchmark']['bias_detected']:
        detections.append('Benchmark')
    if dr['isolation_forest']['kappa'] > 0.20:
        detections.append('IsolationForest')
    if dr['land_corr']['pearson_r'] < 0.40:
        detections.append('LandCorr')
    det_str = ', '.join(detections) if detections else 'NONE'
    print(f'   Bias {bl}: detected by [{det_str}]')

# Fallback summary
print('\n3. FALLBACK CORRECTION:')
for bl in BIAS_LABELS:
    fb = fallback_results[bl]
    best = 'A' if abs(fb['fallback_a']['bias']) < abs(fb['fallback_b']['bias']) else 'B'
    best_bias = fb[f'fallback_{best.lower()}']['bias']
    std_bias = fb['standard']['bias']
    reduction = (1 - abs(best_bias) / abs(std_bias)) * 100 if abs(std_bias) > 0 else 0
    print(f'   Bias {bl}: Best fallback = {best}, '
          f'reduces bias by {reduction:.0f}% '
          f'(${std_bias:+,.0f} -> ${best_bias:+,.0f})')

print('\n4. RECOMMENDATIONS:')
print('   a) ALWAYS run defense layers before trusting FMV predictions')
print('   b) If benchmark check fires (>20% above ceiling), activate fallback')
print('   c) Newer-lease anchoring is safest when lease age data is available')
print('   d) Q25 anchoring is a good structural fix when lease age is unreliable')
print('   e) Even with fallbacks, monitor residual bias in production')
print('\n' + '='*80)
print('Simulation 03 complete.')
print('='*80)